In [1]:
import numpy as np
from numpy import genfromtxt
import glob
from keras.preprocessing.sequence import pad_sequences
from sklearn.cross_validation import train_test_split
import keras
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import sys
pd.set_option('display.max_rows', 1000)
import random
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from scipy import stats
import math

C:\Users\Mahdi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Mahdi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def interpolate(df, index_list, col):
    count = len(index_list)
    
    before_index = index_list[0] - 1
    before_val = df.loc[before_index,col]
    
    after_index = index_list[len(index_list)-1] + 1
    after_val = df.loc[after_index,col]
    
    inc_amount = (after_val - before_val) / (count + 1)
    
    for i in range(count):
        update_index = index_list[i]
        update_val = df.loc[update_index-1,col] + inc_amount
        df.loc[update_index,col] = update_val
        
    return df

In [3]:
data_folder = "data\\Protocol\\"
files_path = data_folder + "subject*.dat"

# ====================
#files_path = data_folder + "subject101.dat"
# ====================

df_main = pd.DataFrame(columns=['exp_id','label_id','acc0','acc1','acc2','gyro0','gyro1','gyro2'])

files = glob.glob(files_path)
for file in files:
    df_exp = pd.read_csv(file, header=None, delimiter=r"\s+")
    df_exp_filtered = df_exp.filter([1, 4, 5, 6, 10, 11, 12])
    df_exp_filtered.columns = ['label_id','acc0','acc1','acc2','gyro0','gyro1','gyro2']
    
    exp_id = int(file[23])
    df_exp_filtered['exp_id'] = exp_id
    
    cols=['exp_id','label_id','acc0','acc1','acc2','gyro0','gyro1','gyro2']
    df_exp_filtered = df_exp_filtered[cols]
    
    total_rows = len(df_exp_filtered)
    interpol_cols = ['acc0','acc1','acc2','gyro0','gyro1','gyro2']
    for col in interpol_cols:
        index = 0
        while index < total_rows:
            if math.isnan(df_exp_filtered.loc[index,col]):
                index_list = [index]
                nan_index = index + 1
                while (True):
                    if math.isnan(df_exp_filtered.loc[nan_index,col]):
                        index_list.append(nan_index)
                        nan_index += 1
                    else:
                        break
                df_exp_filtered = interpolate(df_exp_filtered, index_list, col)
            index += 1
    
    df_main = df_main.append(df_exp_filtered, ignore_index = True)

In [4]:
# test interpolate for exp 1
# df_exp_filtered[19190:19199]

In [5]:
split_exp_id = 7
df_continuous = df_main.loc[df_main['exp_id'] >= split_exp_id]
df_continuous = df_continuous.reset_index(drop=True)

df_OVA = df_main.loc[df_main['exp_id'] < split_exp_id]
df_OVA = df_OVA.reset_index(drop=True)
df_OVA

,exp_id,label_id,acc0,acc1,acc2,gyro0,gyro1,gyro2
0,1,0,2.372230,8.600740,3.510480,-0.092217,0.056812,-0.015845
1,1,0,2.188370,8.565600,3.661790,-0.024413,0.047758,0.006474
2,1,0,2.373570,8.601070,3.548980,-0.057976,0.032574,-0.006988
3,1,0,2.074730,8.528530,3.660210,-0.002352,0.032810,-0.003747
4,1,0,2.229360,8.831220,3.700000,0.012269,0.018305,-0.053325
5,1,0,2.299590,8.829290,3.547100,0.003238,0.012260,-0.054474
6,1,0,2.337380,8.829000,3.547670,-0.023704,-0.031518,-0.047883
7,1,0,2.371420,9.055000,3.393470,0.019069,-0.022004,-0.033647
8,1,0,2.339510,9.132510,3.546680,0.024840,-0.053054,-0.011178
9,1,0,2.259660,9.094150,3.430150,0.007627,-0.057760,-0.019075


In [6]:
window_size = 512
window_move_size = 100 #window_size // 2

In [7]:
segmeneted_activities = [ 'N/A',[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[] ]

for i in range(1 , split_exp_id):

    df_exp = df_OVA.loc[df_OVA['exp_id'] == i]
    df_exp = df_exp.reset_index(drop=True)

    index = 0
    continu = True
    while (continu):

        if index >= len(df_exp):
            break

        df_window = df_exp.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break

        window_label_ids = set(df_window['label_id'].values)
        if (len(window_label_ids) > 1) or \
           (len(window_label_ids)==1 and next(iter(window_label_ids))==0):
            index = index + window_move_size
            continue

        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        window_label_id = next(iter(window_label_ids))

        segmeneted_activities[window_label_id].append(na_window)
        index = index + window_move_size

In [8]:
for i in range(1,25):
    segmeneted_activities[i] = np.array(segmeneted_activities[i])

In [9]:
for i in range(1,25):
    print ("Act_id:" , i , " " , segmeneted_activities[i].shape)

Act_id: 1   (1397, 512, 6)
Act_id: 2   (1469, 512, 6)
Act_id: 3   (1360, 512, 6)
Act_id: 4   (1705, 512, 6)
Act_id: 5   (760, 512, 6)
Act_id: 6   (1140, 512, 6)
Act_id: 7   (1280, 512, 6)
Act_id: 8   (0,)
Act_id: 9   (0,)
Act_id: 10   (0,)
Act_id: 11   (0,)
Act_id: 12   (817, 512, 6)
Act_id: 13   (768, 512, 6)
Act_id: 14   (0,)
Act_id: 15   (0,)
Act_id: 16   (1263, 512, 6)
Act_id: 17   (1733, 512, 6)
Act_id: 18   (0,)
Act_id: 19   (0,)
Act_id: 20   (0,)
Act_id: 21   (0,)
Act_id: 22   (0,)
Act_id: 23   (0,)
Act_id: 24   (323, 512, 6)


In [37]:
def get_binary_X_and_Y_subsampled(act_id):
    
    X , Y = get_binary_X_and_Y (act_id)
    
    total_datapoints = X.shape[0]
    plus_datapoints = segmeneted_activities[act_id].shape[0]
    alpha = (total_datapoints - plus_datapoints) / plus_datapoints
    New_X = []
    New_Y = []
   
    for i in range(total_datapoints):
        
        u = np.random.uniform(0,1)
        if (Y[i]==1) or (u < (1/alpha)):
            New_X.append(X[i])
            New_Y.append(Y[i])

    New_X = np.array(New_X)
    New_Y = np.array(New_Y)
    
    return New_X , New_Y

In [38]:
def get_binary_X_and_Y (act_id):

    X = np.concatenate((segmeneted_activities[1], segmeneted_activities[2]))
    X = np.concatenate((           X            , segmeneted_activities[3]))
    X = np.concatenate((           X            , segmeneted_activities[17]))
    X = np.concatenate((           X            , segmeneted_activities[16]))
    X = np.concatenate((           X            , segmeneted_activities[12]))
    X = np.concatenate((           X            , segmeneted_activities[13]))
    X = np.concatenate((           X            , segmeneted_activities[4]))
    X = np.concatenate((           X            , segmeneted_activities[7]))
    X = np.concatenate((           X            , segmeneted_activities[6]))
    X = np.concatenate((           X            , segmeneted_activities[5]))
    X = np.concatenate((           X            , segmeneted_activities[24]))

    ys = ['N/A',None,None,None,None,None,None,None,None,None,None,None, \
          None,None,None,None,None,None,None,None,None,None,None,None,None]
    for i in range(1,25):
        size = segmeneted_activities[i].shape[0]
        if act_id == i:
            y = np.full (size , 1)
        else:
            y = np.full (size , 0)
        ys[i] = y

    Y = np.concatenate((ys[1], ys[2]))
    Y = np.concatenate((  Y  , ys[3]))
    Y = np.concatenate((  Y  , ys[17]))
    Y = np.concatenate((  Y  , ys[16]))
    Y = np.concatenate((  Y  , ys[12]))
    Y = np.concatenate((  Y  , ys[13]))
    Y = np.concatenate((  Y  , ys[4]))
    Y = np.concatenate((  Y  , ys[7]))
    Y = np.concatenate((  Y  , ys[6]))
    Y = np.concatenate((  Y  , ys[5]))
    Y = np.concatenate((  Y  , ys[24]))

    return (X , Y)

In [67]:
def create_model (data_input_shape):

    LSTM_output_dim = 200
    
    model = Sequential()
    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM1', input_shape=data_input_shape))    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM2'))
    #model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM3'))
    
    model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM1', input_shape=data_input_shape))
    
    model.add(Dense(15, activation='sigmoid',name='Hidden_Layer'))
    
    #model.add(Dense(3, activation='softmax', name='Softmax_layer'))
    model.add(Dense(1, activation='sigmoid',name='Dense_out'))
    
    #model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

    return model

In [71]:
def train_binary_classifier(act_id):
    #(X , Y) = get_binary_X_and_Y (act_id)
    (X , Y) = get_binary_X_and_Y_subsampled (act_id)
    
    X , Y = shuffle(X , Y)

    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)
    X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.333)

    data_input_shape = X.shape[1:3]
    model = create_model(data_input_shape)
    print (model.summary()  , "\n")

    patience = 5
    callbacks=[EarlyStopping(monitor='val_loss',patience=patience,verbose=1)]
    history = model.fit(X_train,y_train,
                        validation_data=[X_valid,y_valid],
                        epochs=5, 
                        batch_size=100
                        ,callbacks=callbacks
                       )

    score = model.evaluate(X_test, y_test, verbose=0)
    print('\nTest loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [72]:
#training binary classifier for activity 1 (Lying)
act_id = 1
model_1 = train_binary_classifier(act_id)
model_1.save('model512_1.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1386 samples, validate on 693 samples
Epoch 1/5
1386/1386 [==============================] - 95s 68ms/step - loss: 0.4896 - acc: 0.8038 - val_loss: 0.3596 - val_acc: 0.9062
Epoch 2/5
1386/1386 [==============================] - 92s 66ms/step - loss: 0.2945 - acc: 0.9329 - val_loss: 0.3038 - val_acc: 0.9177
Epoch 3/5
1386/1386 [==============================] - 90s 65ms/step - loss:

In [ ]:
#training binary classifier for activity 2 (Sitting)
act_id = 2
model_2 = train_binary_classifier(act_id)
model_2.save('model512_2.h5')

In [78]:
#training binary classifier for activity 3 (Standing)
act_id = 3
model_3 = train_binary_classifier(act_id)
model_3.save('model512_3.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1352 samples, validate on 676 samples
Epoch 1/5
1352/1352 [==============================] - 107s 79ms/step - loss: 0.7050 - acc: 0.6021 - val_loss: 0.5967 - val_acc: 0.7041
Epoch 2/5
1352/1352 [==============================] - 101s 75ms/step - loss: 0.5869 - acc: 0.7115 - val_loss: 0.5185 - val_acc: 0.7737
Epoch 3/5
1352/1352 [==============================] - 101s 75ms/step - lo

In [79]:
#training binary classifier for activity 17 (Ironing)
act_id = 17
model_17 = train_binary_classifier(act_id)
model_17.save('model512_17.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1730 samples, validate on 864 samples
Epoch 1/5
1730/1730 [==============================] - 139s 80ms/step - loss: 0.5676 - acc: 0.7139 - val_loss: 0.4277 - val_acc: 0.8252
Epoch 2/5
1730/1730 [==============================] - 136s 78ms/step - loss: 0.4561 - acc: 0.8075 - val_loss: 0.5078 - val_acc: 0.8009
Epoch 3/5
1730/1730 [==============================] - 135s 78ms/step - lo

In [80]:
#training binary classifier for activity 16 (Vacuum Cleaning)
act_id = 16
model_16 = train_binary_classifier(act_id)
model_16.save('model512_16.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1271 samples, validate on 635 samples
Epoch 1/5
1271/1271 [==============================] - 103s 81ms/step - loss: 0.7715 - acc: 0.5083 - val_loss: 0.6495 - val_acc: 0.6346
Epoch 2/5
1271/1271 [==============================] - 102s 80ms/step - loss: 0.6315 - acc: 0.6593 - val_loss: 0.6179 - val_acc: 0.6535
Epoch 3/5
1271/1271 [==============================] - 101s 80ms/step - lo

In [75]:
#training binary classifier for activity 12 (Ascending Stairs)
act_id = 12
model_12 = train_binary_classifier(act_id)
model_12.save('model512_12.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 846 samples, validate on 423 samples
Epoch 1/5
846/846 [==============================] - 69s 82ms/step - loss: 0.6001 - acc: 0.6785 - val_loss: 0.5412 - val_acc: 0.7281
Epoch 2/5
846/846 [==============================] - 64s 76ms/step - loss: 0.5287 - acc: 0.7482 - val_loss: 0.4913 - val_acc: 0.7801
Epoch 3/5
846/846 [==============================] - 64s 75ms/step - loss: 0.5011

In [ ]:
#training binary classifier for activity 13 (Descending Stairs)
act_id = 13
model_13 = train_binary_classifier(act_id)
model_13.save('model512_13.h5')

In [81]:
#training binary classifier for activity 4 (Walking)
act_id = 4
model_4 = train_binary_classifier(act_id)
model_4.save('model512_4.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1728 samples, validate on 864 samples
Epoch 1/5
1728/1728 [==============================] - 164s 95ms/step - loss: 0.5793 - acc: 0.6991 - val_loss: 0.4323 - val_acc: 0.8264
Epoch 2/5
1728/1728 [==============================] - 160s 93ms/step - loss: 0.4057 - acc: 0.8316 - val_loss: 0.3918 - val_acc: 0.8519
Epoch 3/5
1728/1728 [==============================] - 167s 97ms/step - lo

In [ ]:
#training binary classifier for activity 7 (Nordic Walking)
act_id = 7
model_7 = train_binary_classifier(act_id)
model_7.save('model512_7.h5')

In [ ]:
#training binary classifier for activity 6 (Cycling)
act_id = 6
model_6 = train_binary_classifier(act_id)
model_6.save('model512_6.h5')

In [ ]:
#training binary classifier for activity 5 (Running)
act_id = 5
model_5 = train_binary_classifier(act_id)
model_5.save('model512_5.h5')

In [ ]:
#training binary classifier for activity 24 (Rope Jumping)
act_id = 24
model_24 = train_binary_classifier(act_id)
model_24.save('model512_24.h5')

In [ ]:
# from keras.models import load_model
# model = load_model('model.h5')

In [ ]:
def predict(X_predict):

    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0]*24

    for i in range(24):
        model = models[i+1]
        if (model is not None):
            pred_probs = model.predict_proba(X_predict)
            scores[i] = pred_probs[0][0]

    threshold = 0.5
    if max(scores) < threshold:
        label_id = 0
    else:
        label_id = scores.index(max(scores)) + 1
    
    return label_id, scores

In [ ]:
models = [None]*25
models[1] = model_1
models[2] = model_2
models[3] = model_3
models[17] = model_17
models[16] = model_16
models[12] = model_12
models[13] = model_13
models[4] = model_4
models[7] = model_7
models[6] = model_6
models[5] = model_5
models[24] = model_24

# =================
#df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 7]
df_continuous2 = df_continuous
# =================

index = 0
continu = True
correct_preds = 0
total_preds = 0
while (continu):

    if index >= len(df_continuous2):
        break

    df_window = df_continuous2.loc[index:index+window_size-1]
    if len(df_window) < window_size:
        break

    mode_win_labels = stats.mode(df_window['label_id'].values)
    window_label_id = mode_win_labels[0][0]

    if window_label_id == 0:
        print ("Window Label: 0")
        index = index + window_move_size
        continue

    na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
    pred_label_id, _ = predict(na_window)

    print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
    total_preds += 1

    if (pred_label_id == window_label_id):
        correct_preds += 1

    index = index + window_move_size
            
print ("\nContinuous Accuracy: " + str(correct_preds/total_preds))

In [82]:
models = [None]*25
models[1] = model_1
models[2] = model_2
models[3] = model_3
models[17] = model_17
models[16] = model_16
models[12] = model_12
models[13] = model_13
models[4] = model_4
models[7] = model_7
models[6] = model_6
models[5] = model_5
models[24] = model_24

# =================
#df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 7]
df_continuous2 = df_continuous
# =================

index = 0
continu = True
correct_preds = 0
total_preds = 0
while (continu):

    if index >= len(df_continuous2):
        break

    df_window = df_continuous2.loc[index:index+window_size-1]
    if len(df_window) < window_size:
        break

    mode_win_labels = stats.mode(df_window['label_id'].values)
    window_label_id = mode_win_labels[0][0]

    if (window_label_id == 0) or (window_label_id != 12):
        print ("Window Label: " + str(window_label_id))
        index = index + window_move_size
        continue

    na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
    pred_label_id, _ = predict(na_window)

    print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
    total_preds += 1

    if (pred_label_id == window_label_id):
        correct_preds += 1

    index = index + window_move_size
            
print ("\nContinuous Accuracy: " + str(correct_preds/total_preds))

Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window L

Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window Label: 3
Window L

Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 16
Window Label: 

Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 7
Window Label: 12  Predicted Label: 7
Window Label: 12  Predicted Label: 7
W

Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window Label: 4
Window L

...

Window Label: 12  Predicted Label: 5
Window Label: 12  Predicted Label: 5
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 5
Window Label: 12  Predicted Label: 5
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
Window Label: 12  Predicted Label: 4
W

Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window L

Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window L

Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window Label: 7
Window L

Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 24
Window Label: 